Run this file once you have some saved model.

The class defined here wraps *one* trained model.

In [12]:
import numpy as np
import torch
import os

import utils
from load_assoc import ProcessData

In [13]:
class MT_Analyzer:
    def __init__(self, expt_name, disease_class):
        '''dirname should be the entire relative path of the pickled files'''
        self.expt_name = expt_name
        self.disease_class = disease_class
        self.load_models()

    def load_models(self):
        dirname = os.path.join('tensorboard_runs', self.expt_name, 'GEO_GCN_Decagon_feats_')
        self.base_model = torch.load(os.path.join(dirname, 'base_model'),
            map_location=torch.device('cpu'))
        self.state_dicts = []
        for file in os.listdir(dirname):
            if 'model_' not in file: continue
            fold_num = file[-1]
            self.state_dicts.append((fold_num, torch.load(os.path.join(dirname, file))))
        # sort according to fold, as the listdir is not neceesarily alphabetical
        self.state_dicts = [item[1] for item in self.state_dicts]
    
    def load_masks(self, processed_data):
        selected_diseases = processed_data.get_disease_class_idx(self.disease_class)
        curr_y = processed_data.Y.iloc[:, selected_diseases]
        self.masks = []
        for col in curr_y:
            y = torch.tensor(curr_y[col].values.astype('int'), dtype=torch.long)
            self.masks.append(utils.load_masks(y))

    def scores(self, data, fold_num):
        self.base_model.load_state_dict(self.state_dicts[fold_num])
        return self.base_model(data)
    
    def top_k(self, data, fold_num, k=100):
        scores = self.scores(data, fold_num)
        order = np.argsort(self.scores)
        self.top_k = order[-k:]

In [14]:
# load graph
processed_data = ProcessData('../dataset_collection/PP-Decagon_ppi.csv', use_features=True)
X = processed_data.X
X = torch.tensor(X.values, dtype=torch.float)
edges = processed_data.get_edges()
edges = torch.tensor(edges.values, dtype=torch.long)
data = utils.load_graph(X, edges, edge_attr=None)

KeyboardInterrupt: 

In [ ]:
# Get scores from MTL
model = 